In [2]:
import numpy as np
import tensorflow as tf
import pickle
import random
from matplotlib import pyplot as plt
import math
from PIL import Image
from matplotlib.pyplot import imshow

In [3]:
# pickle_file = 'SVHN_multi.pickle'

# with open(pickle_file, 'rb') as f:
#     save = pickle.load(f)
#     svhn_train_dataset = save['train_dataset']
#     svhn_train_labels = save['train_labels']
#     svhn_valid_dataset = save['valid_dataset']
#     svhn_valid_labels = save['valid_labels']
#     svhn_test_dataset = save['test_dataset']
#     svhn_test_labels = save['test_labels']
#     del save  # hint to help gc free up memory
#     print('Training set', svhn_train_dataset.shape, svhn_train_labels.shape)
#     print('Validation set', svhn_valid_dataset.shape, svhn_valid_labels.shape)
#     print('Test set', svhn_test_dataset.shape, svhn_test_labels.shape)
    
    
# svhn_train_labels = (svhn_train_labels.astype(np.int32))
# svhn_test_labels = (svhn_test_labels.astype(np.int32))
# svhn_valid_labels = (svhn_valid_labels.astype(np.int32))

In [4]:
def acc(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])

In [ ]:
# Convert all images to greyscale

In [ ]:
graph_svhn = tf.Graph()

with graph_svhn.as_default():
    HEIGHT = 32
    WIDTH = 32*3
    
    X = tf.placeholder(tf.float32, [None, HEIGHT, WIDTH, 1])
    Y_ = tf.placeholder(tf.int32, [None, 6])
    
    # Learning Rate - alpha
    alpha = tf.placeholder(tf.float32)
    
    # Dropout Probablity
    pkeep = tf.placeholder(tf.float32)
    
    # 5 Layers and their no of neurons
    # 3 Convolutional Layers and a fully connected layer
    K = 12     # First Conv Layer with depth 6
    L = 24     # Second Conv Layer with depth 12
    M = 48    # Third Conv layer with depth 24
    N = 300   # Fourth Fully Connected layer with 200 neurons
    # Last one will be softmax layer with 10 output channels
    
    W1 = tf.Variable(tf.truncated_normal([6, 6, 1, K], stddev=0.1))    # 6x6 patch, 1 input channel, K output channels
    B1 = tf.Variable(tf.constant(0.1, tf.float32, [K]))
    
    W2 = tf.Variable(tf.truncated_normal([5, 5, K, L], stddev=0.1))
    B2 = tf.Variable(tf.constant(0.1, tf.float32, [L]))
    
    W3 = tf.Variable(tf.truncated_normal([4, 4, L, M], stddev=0.1))
    B3 = tf.Variable(tf.constant(0.1, tf.float32, [M]))
    
    W5_1 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1))
    B5_1 = tf.Variable(tf.constant(0.1, tf.float32, [11]))
    
    W5_2 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1))
    B5_2 = tf.Variable(tf.constant(0.1, tf.float32, [11]))
    
    W5_3 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1))
    B5_3 = tf.Variable(tf.constant(0.1, tf.float32, [11]))
    
    W5_4 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1))
    B5_4 = tf.Variable(tf.constant(0.1, tf.float32, [11]))
    
    W5_5 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1))
    B5_5 = tf.Variable(tf.constant(0.1, tf.float32, [11]))
    
    # Model
    stride = 1  # output is 28x140
    Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
    
    stride = 2  # output is 14x70
    Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
    
    stride = 2  # output is 7x35
    Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)

    # reshape the output from the third convolution for the fully connected layer
    shape = Y3.get_shape().as_list()
    YY = tf.reshape(Y3, shape=[-1, shape[1] * shape[2] * shape[3]])
    
    W4 = tf.Variable(tf.truncated_normal([shape[1] * shape[2] * shape[3], N], stddev=0.1))
    B4 = tf.Variable(tf.constant(0.1, tf.float32, [N]))

    Y4 = tf.nn.relu(tf.matmul(YY, W4) + B4)
    YY4 = tf.nn.dropout(Y4, pkeep)
    
    Ylogits_1 = tf.matmul(YY4, W5_1) + B5_1
    Ylogits_2 = tf.matmul(YY4, W5_2) + B5_2
    Ylogits_3 = tf.matmul(YY4, W5_3) + B5_3
    Ylogits_4 = tf.matmul(YY4, W5_4) + B5_4
    Ylogits_5 = tf.matmul(YY4, W5_5) + B5_5   
    ## ('Ylogits_1 shape : ', [None, 11])
    
    Y_1 = tf.nn.softmax(Ylogits_1)
    Y_2 = tf.nn.softmax(Ylogits_2)
    Y_3 = tf.nn.softmax(Ylogits_3)
    Y_4 = tf.nn.softmax(Ylogits_4)
    Y_5 = tf.nn.softmax(Ylogits_5)
  
    cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_1, Y_[:,1])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_2, Y_[:,2])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_3, Y_[:,3])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_4, Y_[:,4])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_5, Y_[:,5]))

    train_prediction = tf.pack([Y_1, Y_2, Y_3, Y_4, Y_5])
    
    train_step = tf.train.AdamOptimizer(alpha).minimize(cross_entropy)
    
    model_saver = tf.train.Saver()

In [ ]:
num_steps_1 = 50001
batch_size = 128

with tf.Session(graph=graphaa) as session:
    tf.initialize_all_variables().run() 
    print('Initialized')
    
    for step in range(num_steps_1):
        #  learning rate decay
        max_learning_rate = 0.003
        min_learning_rate = 0.0001
        decay_speed = 2000.0
        learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-step/decay_speed)

        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = svhn_train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = svhn_train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {X : batch_data, Y_ : batch_labels, pkeep : 0.80, alpha : learning_rate}
        _, l, train_pred = session.run([train_step, cross_entropy, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0): 
            print('Loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % acc(train_pred, batch_labels[:,1:6]))
            print('Learning rate : ', learning_rate)
            print('    ')
            
    print('Training Complete on SVHN Data')
    
    save_path = model_saver.save(session, "saved_models/CNN_SVHN_New.ckpt")
    print("Model saved in file: %s" % save_path)
    
#     _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : svhn_test_dataset, Y_ : svhn_test_labels, pkeep : 1.0, alpha : 0.002})
#     print('Test accuracy: ', acc(predictions, svhn_test_labels[:,1:6]))
    
#     _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : svhn_valid_dataset, Y_ : svhn_valid_labels, pkeep : 1.0, alpha : 0.002})
#     print('Validation accuracy: ', acc(predictions, svhn_valid_labels[:,1:6]))

In [ ]:
with tf.Session(graph=graphaa) as session:
    model_saver.restore(session, "saved_models/CNN_SVHN_New.ckpt")
    print("Model restored.") 
    print('Initialized')
    
    _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : svhn_test_dataset, Y_ : svhn_test_labels, pkeep : 1.0, alpha : 0.002})
    print('Test accuracy: ', acc(predictions, svhn_test_labels[:,1:6]))
    
#     _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : svhn_valid_dataset, Y_ : svhn_valid_labels, pkeep : 1.0, alpha : 0.002})
#     print('Validation accuracy: ', acc(predictions, svhn_valid_labels[:,1:6]))